In [148]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

def get_date(soup):
    value = soup.find('p',{'class':'sub'}).text.strip() + ', ' + str(datetime.now().year)
    if value:
        return value
    else:
        return "N/A"

def get_updated_time(soup):
    value = soup.find('p',{'class':'cur-con-weather-card__subtitle'}).text.strip()
    if value:
        return value
    else:
        return "00:00 AM"

def get_loction(soup):
    value = soup.find('h1',{'class':'header-loc'}).text.strip()
    if value:
        return value
    else:
        return "N/A"
    
def get_temperature(soup):
    value = soup.find('div',{'class':'temp'}).text.strip()
    if value:
        return value
    else:
        return "N/A"

def get_condition(soup):
    value = soup.find('span',{'class':'phrase'}).text.strip()
    if value:
        return value
    else:
        return "N/A"
    
def get_realfeel_shade(soup):
    detail_divs = soup.find_all('div',{'class':'spaced-content detail'})
    for div in detail_divs:
        label = div.find("span", {'class':'label'})
        value = div.find("span", {'class':'value'})
        label_text = label.text.strip().lower()
        value_text = value.text.strip()
        # Match and append based on keys
        if 'realfeel shade™' in label_text:
            return value_text
    return "N/A"

def get_wind(soup):
    detail_divs = soup.find_all('div',{'class':'spaced-content detail'})
    for div in detail_divs:
        label = div.find("span", {'class':'label'})
        value = div.find("span", {'class':'value'})
        label_text = label.text.strip().lower()
        value_text = value.text.strip()
        # Match and append based on keys
        if 'wind' in label_text:
            return value_text
    return "N/A"

def get_air_quality(soup):
    detail_divs = soup.find_all('div',{'class':'spaced-content detail'})
    for div in detail_divs:
        label = div.find("span", {'class':'label'})
        value = div.find("span", {'class':'value'})
        label_text = label.text.strip().lower()
        value_text = value.text.strip()
        # Match and append based on keys
        if 'air quality' in label_text:
            return value_text
    return "N/A"


if __name__ == '__main__':

    #By adding the user agent string, you fool it into thinking that your script is a browser making a request.
    Headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36',}

    #The webpage URL
    URL = 'https://www.accuweather.com/'

    #HTTP request
    html_data = requests.get(URL,headers=Headers).text

    #Soup object containing all data
    soup = BeautifulSoup(html_data,'html.parser')

    #Get link for all locations
    Location_a_tags = soup.find_all('a',{'class':'nearby-location weather-card'})
    Location_Links = []
    for link in Location_a_tags:
        Location_Links.append(URL+link['href'])

    #Pre-defined dictionary
    Dict = {'Date':[],'Updated_Time':[],'Location':[],'Temperature':[],'RealFeel_Shade':[],'Wind':[],'Air_Quality':[],'Condition':[]}

    #For each location, get all details
    for link in Location_Links:

        #HTTP request for a location
        new_html_data = requests.get(link,headers=Headers).text
        new_soup = BeautifulSoup(new_html_data,'html.parser')

        #append details
        Dict['Date'].append(get_date(new_soup))
        Dict['Updated_Time'].append(get_updated_time(new_soup))
        Dict['Location'].append(get_loction(new_soup))
        Dict['Temperature'].append(get_temperature(new_soup))
        Dict['Condition'].append(get_condition(new_soup))
        Dict['RealFeel_Shade'].append(get_realfeel_shade(new_soup))
        Dict['Wind'].append(get_wind(new_soup))
        Dict['Air_Quality'].append(get_air_quality(new_soup))

    #Dictionary to Dataframe
    df = pd.DataFrame.from_dict(Dict)

    #Loading extracted data to csv
    df.to_csv("weather_data.csv", header=True, index=False)